# Predicción de la obesidad

## Este cuaderno esta realizado con jupyter notebook el 26 de Enero del 2024 y la información esta actualizada hasta el 25 de Enero del 2024

### El cuaderno esta realizado por José de Jesús Hernández Vázquez

#### Los conjuntos de datos estan todos en kaggle.omc con el siguiente link: https://www.kaggle.com/datasets/mrsimple07/obesity-prediction/data

#### El conjunto de datos proporciona información completa sobre las características demográficas, los atributos físicos y los hábitos de estilo de vida de los individuos, con el objetivo de facilitar el análisis y la predicción de la prevalencia de la obesidad. Incluye variables clave como edad, sexo, altura, peso, índice de masa corporal (IMC), nivel de actividad física y categoría de obesidad, lo que ofrece información valiosa sobre los factores que influyen en los resultados de la obesidad.

#### Para este cuadernillo vamos a investigar lo siguiente:
1. Preprocesamiento de datos
2. Definición del modelo 
3. Preparación del modelo
4. Entrenamiento del modelo
5. Validación del modelo
6. Predicciones

### Vamos a hacer las importaciones necesarias para poder realizar este proyecto¶


In [273]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pandas as pd
import matplotlib.pyplot as plt

### En este caso solo hay un conjunto de datos, por lo cual vamos a cargarlo directamente en una variable

In [274]:
df_train = pd.read_csv('./assets/obesity_data.csv')

In [277]:
df_train

,Age,Gender,Height,Weight,BMI,PhysicalActivityLevel,ObesityCategory
0,56,Male,173.575262,71.982051,23.891783,4,Normal weight
1,69,Male,164.127306,89.959256,33.395209,2,Obese
2,46,Female,168.072202,72.930629,25.817737,4,Overweight
3,32,Male,168.459633,84.886912,29.912247,3,Overweight
4,60,Male,183.568568,69.038945,20.487903,3,Normal weight
...,...,...,...,...,...,...,...
995,18,Male,155.588674,64.103182,26.480345,4,Overweight
996,35,Female,165.076490,97.639771,35.830783,1,Obese
997,49,Female,156.570956,78.804284,32.146036,1,Obese
998,64,Male,164.192222,57.978115,21.505965,4,Normal weight


### Una vez que tenemos nuestras variables separadas, tomando en cuenta que el genero es la unica variable no numerica, vamos a convertirla para poder hacerlo de manera eficiente

In [278]:
# Codificar la variable categórica "Gender"
gender_encoder = LabelEncoder()
df_train['Gender'] = gender_encoder.fit_transform(df_train['Gender'])

In [300]:
df_train

,Age,Gender,Height,Weight,BMI,PhysicalActivityLevel,ObesityCategory
0,56,1,173.575262,71.982051,23.891783,4,Normal weight
1,69,1,164.127306,89.959256,33.395209,2,Obese
2,46,0,168.072202,72.930629,25.817737,4,Overweight
3,32,1,168.459633,84.886912,29.912247,3,Overweight
4,60,1,183.568568,69.038945,20.487903,3,Normal weight
...,...,...,...,...,...,...,...
995,18,1,155.588674,64.103182,26.480345,4,Overweight
996,35,0,165.076490,97.639771,35.830783,1,Obese
997,49,0,156.570956,78.804284,32.146036,1,Obese
998,64,1,164.192222,57.978115,21.505965,4,Normal weight


In [279]:
features = df_train[['Age', 'Gender', 'Height', 'Weight', 'BMI', 'PhysicalActivityLevel']]

# Seleccionamos nuestra variable que vamos a predecir

labels = df_train['ObesityCategory']


In [281]:
# Codificar las etiquetas como números
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)


### Vamos a dividir el conjunto de datos en entrenamiento y prueba

In [282]:
X_train, X_test, y_train, y_test = train_test_split(features, labels_encoded, test_size=0.2, random_state=42)

### Escalaremos las caracteristicas para mejorar el rendimiento del modelo

In [283]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)


### Convertimos los datos en tensores de Pytorch, ya que en este caso vamos a utlilizar Pytorch para predecir nuestro modelo

In [284]:
X_train_tensor = torch.FloatTensor(X_train_scaled)
y_train_tensor = torch.LongTensor(y_train)
X_test_tensor = torch.FloatTensor(X_test_scaled)
y_test_tensor = torch.LongTensor(y_test)


### Una vez que ya tenemos todos nuestros datos preparados podemos continuar con la parte importante de nuestro problema

# Modelo de predicción

## Definición del modelo

In [285]:
class ObesityModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ObesityModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)

        return x

## Preparación del modelo

In [286]:
input_size = X_train_tensor.shape[1] ### Numero de caracteristicas
hidden_size = 64 
output_size = len(label_encoder.classes_) #Número de clases

model = ObesityModel(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss() # Lo utilizamos para un problema de clasificación
optimizer = optim.Adam(model.parameters(), lr = 0.001)

## Entrenamiento del modelo

In [287]:
num_epochs = 100

for epoch in range(num_epochs):
    optimizer.zero_grad()

    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    loss.backward()
    optimizer.step()

    

## Validación del modelo

In [288]:
with torch.no_grad():
    model.eval()
    test_outputs = model(X_test_tensor)
    test_loss = criterion(test_outputs, y_test_tensor)
    print(f'Perdida del ensayo: {test_loss.item()}')

Perdida del ensayo: 0.7762184143066406


#### Como podemos observar obtuvimos un valor de 0.77, en si determinar si este es un valor bueno o malo podria depender de varios factores según el contexto del mismo problema

# Predicciones

In [291]:
X = df_train.drop('ObesityCategory', axis = 1)

X.head()

,Age,Gender,Height,Weight,BMI,PhysicalActivityLevel
0,56,1,173.575262,71.982051,23.891783,4
1,69,1,164.127306,89.959256,33.395209,2
2,46,0,168.072202,72.930629,25.817737,4
3,32,1,168.459633,84.886912,29.912247,3
4,60,1,183.568568,69.038945,20.487903,3


## Vamos a realizar un procesamiento totalmente similar al previo, transformando nuestro dataframe en tensores

In [292]:
X_tensor = torch.FloatTensor(scaler.transform(X))

## Vamos a hacer una evaluación del modelo

In [293]:
model.eval()

with torch.no_grad():
    predictions = model(X_tensor)

## Aplicamos la función de softmax para obtener probabilidades

In [294]:
prediction_probs = torch.softmax(predictions, dim = 1)

## Obtenemos la clase predicha para cada ejemplo

In [295]:
predicted_classes = torch.argmax(prediction_probs, dim = 1)

## Decodificamos las clases predichas (usando label_encoder)

In [296]:
predicted_labels = label_encoder.classes_[predicted_classes.numpy()]

## Revertimos la codificación numérica de "Gender" usando inverse_transform

In [301]:
X_with_predictions = X.copy()
X_with_predictions['Predicted_Obesity_Category'] = predicted_labels
X_with_predictions['Gender'] = gender_encoder.inverse_transform(X_with_predictions['Gender'])

## Realizamos una comparativa de nuestro modelo con respecto al dataset original

In [303]:
X_with_predictions

,Age,Gender,Height,Weight,BMI,PhysicalActivityLevel,Predicted_Obesity_Category
0,56,Male,173.575262,71.982051,23.891783,4,Normal weight
1,69,Male,164.127306,89.959256,33.395209,2,Obese
2,46,Female,168.072202,72.930629,25.817737,4,Overweight
3,32,Male,168.459633,84.886912,29.912247,3,Obese
4,60,Male,183.568568,69.038945,20.487903,3,Normal weight
...,...,...,...,...,...,...,...
995,18,Male,155.588674,64.103182,26.480345,4,Overweight
996,35,Female,165.076490,97.639771,35.830783,1,Obese
997,49,Female,156.570956,78.804284,32.146036,1,Obese
998,64,Male,164.192222,57.978115,21.505965,4,Normal weight


In [299]:
df_train

,Age,Gender,Height,Weight,BMI,PhysicalActivityLevel,ObesityCategory
0,56,1,173.575262,71.982051,23.891783,4,Normal weight
1,69,1,164.127306,89.959256,33.395209,2,Obese
2,46,0,168.072202,72.930629,25.817737,4,Overweight
3,32,1,168.459633,84.886912,29.912247,3,Overweight
4,60,1,183.568568,69.038945,20.487903,3,Normal weight
...,...,...,...,...,...,...,...
995,18,1,155.588674,64.103182,26.480345,4,Overweight
996,35,0,165.076490,97.639771,35.830783,1,Obese
997,49,0,156.570956,78.804284,32.146036,1,Obese
998,64,1,164.192222,57.978115,21.505965,4,Normal weight


#### Apartir de este punto, nos damos cuenta que aunque no todos los datos son exactamente iguales, el modelo tiene una precisión bastante acercada lo cual nos demuestra que es un modelo bueno y funcional, podriamos seguir mejorandolo si decidimos hacer más entrenamiento sobre el modelo